In [11]:
import torch as t
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
import torchvision.models as models
import torchvision.transforms as transforms
import torchvision.datasets as datasets
from torch.utils.data import Dataset, DataLoader, random_split
from sklearn.model_selection import train_test_split
from datasets import load_dataset 
from PIL import Image
import os, random
import pandas as pd
import transformers as tr
import json
import shutil

In [2]:
# DO NOT RUN CELL!

"""lines = []
with open("hateful_memes/train.jsonl") as f:
    lines = f.read().splitlines()

lines_dict = [json.loads(line) for line in lines]

df = pd.DataFrame(lines_dict)
print(df.loc[[0]])"""

      id            img  label  \
0  42953  img/42953.png      0   

                                               text  
0  its their character not their color that matters  


In [12]:
# DO NOT RUN CELL!
'''
imgs_referenced = df["img"]
imgs_referenced.astype(str)

def select_ref_imgs(orig_loc):
    orig_loc = str(orig_loc)
    file_name = orig_loc[4:]
   
    try:
        shutil.move(f".\hateful_memes\{orig_loc}", f".\hateful_memes\imgs_present/{file_name}")
        print(f"file {file_name} moved")
    except FileNotFoundError:
        print(f"file {file_name} not found")
        pass
    except Exception as e:
        print(f"error:{e}")

def change_slash(string):
    string = string.replace("/",  "\ " )
    string = string.replace(" ", "")
    return string
    '''

# imgs_referenced = imgs_referenced.apply(change_slash)
# imgs_referenced.apply(select_ref_imgs)

print("files moved")

files moved


In [13]:
# DO NOT RUN CELL!

# for file_name in os.listdir("./hateful_memes/img"):
    # file_path = os.path.join("./hateful_memes/img", file_name)
    # os.remove(file_path)
    # print(f"{file_path} deleted.")
    # pass
print ("files deleted")

files deleted


In [ ]:
# DO NOT RUN CELL!

# shutil.rmtree("hateful_memes/img_old")

In [7]:
# DO NOT RUN CELL!

# os.rename("hateful_memes/imgs_present", "hateful_memes/img")

In [14]:
# DO NOT RUN CELL!

"""def check_imgs_exist(file_path):
    file_path = str(file_path)
    file_name = file_path[4:]
    if os.path.isfile(f"hateful_memes/{file_path}"):
        # print(f"{file_name} present")
        return True
    else:
        return False
    
reference_df = df
reference_df["img_exists"] = reference_df["img"].apply(check_imgs_exist)
reference_df = reference_df[reference_df["img_exists"]]
reference_df = reference_df.drop(columns=["img_exists"])
reference_df = reference_df.reset_index()
"""

files present


In [4]:
# DO NOT RUN CELL!

# print(reference_df.head(), len(reference_df))

   index     id            img  label  \
0      0  42953  img/42953.png      0   
1      2  13894  img/13894.png      0   
2      4  82403  img/82403.png      0   
3      5  16952  img/16952.png      0   
4      6  76932  img/76932.png      0   

                                                text  
0   its their character not their color that matters  
1                           putting bows on your pet  
2  everybody loves chocolate chip cookies, even h...  
3           go sports! do the thing! win the points!  
4     fine you're right. now can we fucking drop it?   6744


In [ ]:
# DO NOT RUN CELL!

# print(reference_df.query("id == '14507'"))

     index     id            img  label  \
882   1112  14507  img/14507.png      1   

                                                  text  
882  when your history teacher starts talking about...  


In [15]:
# DO NOT RUN CELL!

# reference_df = reference_df.astype({"index":int, "id":int, "img":str, "label":float, "text":str})
# data = reference_df

# data.to_csv("data.csv")

In [2]:
data = pd.read_csv("data.csv")
print (data.head())

   Unnamed: 0  index     id            img  label  \
0           0      0  42953  img/42953.png    0.0   
1           1      2  13894  img/13894.png    0.0   
2           2      4  82403  img/82403.png    0.0   
3           3      5  16952  img/16952.png    0.0   
4           4      6  76932  img/76932.png    0.0   

                                                text  
0   its their character not their color that matters  
1                           putting bows on your pet  
2  everybody loves chocolate chip cookies, even h...  
3           go sports! do the thing! win the points!  
4     fine you're right. now can we fucking drop it?  


In [102]:
class DatasetTxt (Dataset):
    def __init__(self, df, tokenizer):
        super().__init__()
        self.df = df
        self.tokenizer = tokenizer

        self.text= df["text"].values
        self.label = df["label"].values

    def __len__(self):
        return len(self.df["label"])
    
    def __getitem__(self, index):
        text = self.text[index]
        label = self.label[index]

        encoding = self.tokenizer.encode_plus(text, add_special_tokens=True, return_token_type_ids=False,
            padding='longest', return_attention_mask=True, return_tensors='pt', truncation=True)

        return {"input_ids": encoding["input_ids"].flatten(), "attention_mask": encoding["attention_mask"].flatten(),
                "labels": t.tensor(label, dtype=t.long)}
        
class DatasetImg (Dataset):
    def __init__(self, df, img_dir):
        super().__init__()
        self.df = df
        self.img_dir = img_dir
        self.transform = transforms.Compose([
                    transforms.Resize((224, 224)),
                    transforms.ToTensor(),
                    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
                    ])

        self.img= df["img"].values
        self.label = df["label"].astype(int).values

    def __len__(self):
        return len(self.df["label"])
    
    def __getitem__(self, index):
        img_pth = os.path.join(self.img_dir, self.img[index])
        img = Image.open(img_pth).convert("RGB")
        img_tensor = self.transform(img).unsqueeze(0)
        img_tensor = img_tensor.squeeze(1)
        label = self.label[index]

        return img_tensor, label



In [20]:
# splitting data 50%
train_data, rest = train_test_split(data, test_size= 0.5)

In [13]:
tokenizer = tr.BertTokenizer.from_pretrained("Hate-speech-CNERG/dehatebert-mono-english", clean_up_tokenization_spaces=True)

In [36]:
text_ds = DatasetTxt(train_data, tokenizer=tokenizer)

In [103]:
imag_ds = DatasetImg(train_data, img_dir="hateful_memes/")

In [15]:
d_collator = tr.DataCollatorWithPadding(tokenizer=tokenizer)

In [104]:
train_loader_txt = DataLoader(text_ds, batch_size=16, shuffle=True, collate_fn=d_collator)
train_loader_img = DataLoader(imag_ds, batch_size=16, shuffle=True)

In [105]:
class MonoLateFusModel(nn.Module):
    def __init__(self, text_model, img_model, text_dims, img_dims):
        super().__init__()
        self.text_model = text_model
        self.img_model = img_model


        # architecture
        #ToxicBERT/HateBERT input
        self.textfc = nn.Linear(text_dims, 128)
        #ResNet input
        self.imgfc = nn.Linear(img_dims, 128)
        self.mlp = nn.Sequential(nn.Linear(128, 16), nn.ReLU(), nn.Linear(16, 2))

    def forward(self, tokenized_text, image):
        #to be done

        txt_proc = self.textfc(self.text_model(tokenized_text))
        img_proc = self.imgfc(self.img_model(image))

        txt_fft = t.fft.fft(txt_proc)
        img_fft = t.fft.fft(img_proc)

        fusion = t.real(txt_fft * img_fft)
        logits = self.mlp(fusion)
        # combined_outputs = t.cat([txt_proc, img_proc], 1)
        # outputs = self.fusion(combined_outputs)

        return logits
    
    def fine_tune_subpart(self, dataloader, input_type):
        model = self.text_model if input_type == "txt" else self.img_model
        model.train()


        tot_loss = 0
        correct_preds = 0
        if input_type == "txt":
            for name, param in model.named_parameters():
                if "classifier" not in name:
                    param.requires_grad = False
            optimizer_txt = optim.AdamW(model.parameters(), lr= 5e-5)
            scheduler = optim.lr_scheduler.StepLR(optimizer_txt, step_size=1, gamma=0.9)
        elif input_type == "img":
            for name, param in model.named_parameters():
                if "fc" not in name:
                    param.requires_grad = False
            criterion = nn.CrossEntropyLoss()
            optimizer_img = optim.Adam(model.fc.parameters(), lr=0.001)

        for e in range(3):
            if input_type == "txt":
                for batch in dataloader:
                    toktext = batch["input_ids"]
                    mask = batch["attention_mask"]
                    labels = batch["labels"]


                    outputs = model(input_ids=toktext, attention_mask=mask, labels=labels)

                    # loss= nn.CrossEntropyLoss()

                    loss = outputs.loss
                    logits = outputs.logits

                    optimizer_txt.zero_grad()
                    loss.backward()
                    optimizer_txt.step()
                    scheduler.step()
                    tot_loss += loss.item()

                    pred = t.argmax(logits, -1)
                    

            elif input_type == "img":
                for images, labels in dataloader:

                    images = images.squeeze(1)

                    optimizer_img.zero_grad()
                    outputs = model(images)
                    loss = criterion(outputs.float(), labels)
                    loss.backward()
                    optimizer_img.step()
                    tot_loss += loss.item()
                    
                    _, pred = t.max(outputs.data, 1)
            
            else:
                print ("input_type must be either 'txt' or 'img'")
                
            correct_preds = t.sum(pred == labels)
            print(f"batch {e+1}/3", 
                  f"loss: {tot_loss/(len(dataloader)*(e+1)):.4f}, accuracy: {correct_preds/(len(dataloader)*(e+1)):.4f}")
        return "loss:", tot_loss/len(dataloader), "accuracy:", correct_preds/len(dataloader)

In [19]:
model_txt = tr.AutoModelForSequenceClassification.from_pretrained("Hate-speech-CNERG/dehatebert-mono-english", num_labels = 2)
model_img = models.resnet18(weights=models.ResNet18_Weights.IMAGENET1K_V1)

In [106]:
hmc_lf_model = MonoLateFusModel(text_model=model_txt, img_model=model_img, text_dims=768, img_dims=2048)

In [64]:
hmc_lf_model.fine_tune_subpart(train_loader_txt, "txt")

batch1/3 loss: 0.6901544030243751, accuracy: 0.033175356686115265
batch2/3 loss: 1.3740130353595408, accuracy: 0.028436018154025078
batch3/3 loss: 2.061435442329583, accuracy: 0.0379146933555603


('loss:', 2.061435442329583, 'accuracy:', tensor(0.0379))

In [107]:
hmc_lf_model.fine_tune_subpart(train_loader_img, "img")

batch 1/3 loss: 1.2518, accuracy: 0.0284
batch 2/3 loss: 0.9980, accuracy: 0.0213
batch 3/3 loss: 0.8919, accuracy: 0.0142


('loss:', 2.6755912041212144, 'accuracy:', tensor(0.0427))

In [39]:
print(hmc_lf_model.text_model)

BertForSequenceClassification(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(105879, 768)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.3, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0-11): 12 x BertLayer(
          (attention): BertAttention(
            (self): BertSdpaSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-12, elementwi